# Varying diameter

In this notebook we will try to create a scaling law to account for a varying propeller diameter as well as a varying number of blade and a varying altitude. Twist law should also vary but for now it will be left as is.

In [ ]:
import warnings

warnings.filterwarnings(action="ignore")

In [ ]:
# This file is part of FAST-OAD_CS23-HE : A framework for rapid Overall Aircraft Design of Hybrid
# Electric Aircraft.
# Copyright (C) 2022 ISAE-SUPAERO

# Import pyVPLM modules/functions
from pyvplm.core.definition import PositiveParameter, PositiveParameterSet
from pyvplm.addon.variablepowerlaw import buckingham_theorem

from stdatm import Atmosphere

altitude_min, altitude_max = 0, 30000  # In feet
rho_min = Atmosphere(altitude_max, altitude_in_feet=True).density
rho_max = Atmosphere(altitude_min, altitude_in_feet=True).density

# Declare physical variables and set (limits have no impact since DOE is not constructed)
# t = PositiveParameter("t", [0.01, 46172.16929583475], "N", "trust")
# rho = PositiveParameter("rho", [rho_min, rho_max], "kg/m^3", "air density")
# n_rot = PositiveParameter("n_rot", [11., 34.0], "Hz", "rotational speed")
# d = PositiveParameter("d", [2.5, 3.1], "m", "propeller diameter")
# pitch = PositiveParameter("pitch", [0.087, 0.87], "rad", "propeller pitch @0.75 of the radius")
# PITCH_REF = PositiveParameter(
#     "PITCH_REF", [0.087], "rad", "propeller reference pitch @0.75 of the radius"
# )
# v = PositiveParameter("v", [5.0, 180.0], "m/s", "air speed")
# c_mid_ratio = PositiveParameter(
#     "c_mid_ratio", [1.1, 5.0], "", "chord at the mid point as a ratio of root chord"
# )
# c_mid_radius_ratio = PositiveParameter(
#     "c_mid_radius_ratio", [0.4, 0.8], "", "position of the mid point chord as a ratio of radius"
# )
# parameter_set = PositiveParameterSet(
#     t, rho, n_rot, d, v, c_mid_ratio
# )

# # Define d and lambda_wind to be first elected as repetitive set
# parameter_set.first("n_rot", "d", "rho")
# # parameter_set.latex_render()

# # Calculate pi parameters with first repetitive set found in parameters' order
# pi_set, _ = buckingham_theorem(parameter_set, False)
# pi_set.latex_render()

We'll adjust slightly the defined set here to take something similar to the previous notebook, that is $\pi_2 = \frac{picth}{PITCH_{REF}}$ and $\pi_3 = n_{blade}$

In [ ]:
# Import pyVPLM modules/functions
# from pyvplm.core.definition import Constraint, ConstraintSet
# from pyvplm.addon.pixdoe import create_const_doe
# from pyvplm.addon.variablepowerlaw import (
#     declare_func_x_to_pi,
#     reduce_parameter_set,
#     declare_constraints,
# )
# from pyvplm.addon.comsoladdon import save_file

# reduced_parameter_set, reduced_pi_set = reduce_parameter_set(
#     parameter_set, pi_set, elected_output="t"
# )
# # reduced_pi_set.latex_render()
# # reduced_parameter_set.latex_render()

# constraint1 = Constraint("n_rot*d>=33")
# parameters_constraints = declare_constraints(reduced_parameter_set, ConstraintSet(constraint1))

# # Save function to translate x parameters to pi parameters
# func_x_to_pi = declare_func_x_to_pi(reduced_parameter_set, reduced_pi_set)

# results = create_const_doe(
#     reduced_parameter_set,
#     reduced_pi_set,
#     func_x_to_pi,
#     whished_size=100,
#     choice_nb=7,
#     log_space=False,
# #     parameters_constraints=parameters_constraints,
# )
# doeX = results[0]

In [ ]:
from pyDOE2 import *
import copy
from stdatm import Atmosphere


doe_temp = lhs(10, samples=2000, criterion="correlation")
doeX = copy.deepcopy(doe_temp)

n_rot_min, n_rot_max = 3000.0 / 60.0, 6000.0 / 60.0

D_min, D_max = 0.5, 1.5

altitude_min, altitude_max = 0, 30000  # In feet
rho_min = Atmosphere(altitude_max, altitude_in_feet=True).density
rho_max = Atmosphere(altitude_min, altitude_in_feet=True).density

v_min, v_max = 5.0, 130.0

pitch_min, pitch_max = 10 * np.pi / 180.0, 50 * np.pi / 180.0

c_mid_radius_val_min, c_mid_radius_val_max = 1.2, 5.0
c_mid_radius_ratio_val_min, c_mid_radius_ratio_val_max = 0.4, 0.8

tip_twist_min, tip_twist_max = 1.0 * np.pi / 180.0, 45.0 * np.pi / 180.0
c_root_radius_val_min, c_root_radius_val_max = 0.025, 0.05

doeX[:, 0] = doe_temp[:, 0] * (1.05 * n_rot_max - 0.95 * n_rot_min) + 0.95 * n_rot_min
doeX[:, 1] = doe_temp[:, 1] * (1.05 * D_max - 0.95 * D_min) + 0.95 * D_min
doeX[:, 2] = doe_temp[:, 2] * (1.05 * rho_max - 0.95 * rho_min) + 0.95 * rho_min
doeX[:, 3] = doe_temp[:, 3] * (1.05 * v_max - 0.95 * v_min) + 0.95 * v_min
doeX[:, 4] = np.full_like(doe_temp[:, 4], 0.087)
doeX[:, 5] = doe_temp[:, 5] * (1.05 * pitch_max - 0.95 * pitch_min) + 0.95 * pitch_min
doeX[:, 6] = (
    doe_temp[:, 6] * (1.05 * c_mid_radius_val_max - 0.95 * c_mid_radius_val_min)
    + 0.95 * c_mid_radius_val_min
)
doeX[:, 7] = (
    doe_temp[:, 7] * (1.05 * c_mid_radius_ratio_val_max - 0.95 * c_mid_radius_ratio_val_min)
    + 0.95 * c_mid_radius_ratio_val_min
)
doeX[:, 8] = doe_temp[:, 8] * (tip_twist_max - tip_twist_min) + tip_twist_min
doeX[:, 9] = (
    doe_temp[:, 9] * (1.05 * c_root_radius_val_max - 0.95 * c_root_radius_val_min)
    + 0.95 * c_root_radius_val_min
)

In [ ]:
import numpy as np

print(np.min(doeX[:, 0]), np.max(doeX[:, 0]))
print(np.min(doeX[:, 1]), np.max(doeX[:, 1]))
print(np.min(doeX[:, 2]), np.max(doeX[:, 2]))
print(np.min(doeX[:, 3]), np.max(doeX[:, 3]))
print(np.min(doeX[:, 4]), np.max(doeX[:, 4]))
print(np.min(doeX[:, 5]), np.max(doeX[:, 5]))
print(np.min(doeX[:, 6]), np.max(doeX[:, 6]))
print(np.min(doeX[:, 7]), np.max(doeX[:, 7]))
print(np.min(doeX[:, 8]), np.max(doeX[:, 8]))
print(np.min(doeX[:, 9]), np.max(doeX[:, 9]))

In [ ]:
def extend_doe(doe, array_to_add, altitude):

    data_len = doe.shape[0]

    first_array_to_add = np.full(data_len, array_to_add[0])
    temp_doeX = np.c_[doe, first_array_to_add]

    doe_X_to_return = copy.deepcopy(temp_doeX)
    altitude_to_return = copy.deepcopy(altitude)

    if len(array_to_add) != 1:
        for element_to_add in array_to_add[1:]:
            element_to_add_array = np.full(data_len, element_to_add)
            temp_doeX[:, -1] = element_to_add_array
            doe_X_to_return = np.r_[doe_X_to_return, temp_doeX]
            altitude_to_return = np.r_[altitude_to_return, altitude]

    return doe_X_to_return, altitude_to_return

In [ ]:
import numpy as np
import copy

from fastga_he.models.propulsion.components.propulsor.propeller.methodology.performance_point import (
    _ComputePropellerPointPerformance,
)

RHO_val_temp = np.array(doeX[:, 2])

# Finding associated altitudes
altitude_orig = np.zeros_like(RHO_val_temp)
altitude_finder = _ComputePropellerPointPerformance()

for idx, rho in enumerate(RHO_val_temp):
    altitude_orig[idx] = altitude_finder.find_altitude(rho)

print(doeX.shape)
print(altitude_orig.shape)

# possible_pitch_ref = np.array([0.087])
# new_doeX, new_altitude = extend_doe(doeX, possible_pitch_ref, altitude_orig)

print(doeX.shape)
print(altitude_orig.shape)

# possible_pitch = np.array([10, 15, 20, 25, 30, 35, 40, 45]) * np.pi / 180.0
# new_doeX, new_altitude = extend_doe(new_doeX, possible_pitch, new_altitude)

print(doeX.shape)
print(altitude_orig.shape)

n_rot_val = np.array(doeX[:, 0])
D_val = np.array(doeX[:, 1])
RHO_val = np.array(doeX[:, 2])
v_val = np.array(doeX[:, 3])
pitch_val = np.array(doeX[:, 5])

atm = Atmosphere(altitude_orig, altitude_in_feet=False)
tip_speed = np.sqrt((n_rot_val * 2 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0 + v_val ** 2.0)

proper_idx_1 = np.where(tip_speed / atm.speed_of_sound <= 0.95)[0]

aoa_75 = pitch_val - np.arctan2(v_val, 0.75 * n_rot_val * (D_val / 2.0) * 2.0 * np.pi)

proper_index_2 = np.where(abs(aoa_75 - 0.05) < 0.27)[0]

proper_index = np.intersect1d(proper_idx_1, proper_index_2)

new_doeX = doeX[proper_index, :]
new_altitude = altitude_orig[proper_index]

print(new_doeX.shape)
print(new_altitude.shape)

possible_blade_number = np.array([2, 3, 4, 5, 6])
possible_c_mid_ratio_val = np.array([1.1, 2.075, 3.05, 4.025, 5.0])
possible_c_mid_radius_ratio_val = np.array([0.4, 0.5, 0.6, 0.7, 0.8])
# new_doeX, new_altitude = extend_doe(new_doeX, possible_c_mid_ratio_val, new_altitude)
# new_doeX, new_altitude = extend_doe(new_doeX, possible_c_mid_radius_ratio_val, new_altitude)
# new_doeX, new_altitude = extend_doe(new_doeX, possible_pitch, altitude_orig[proper_index])
new_doeX, new_altitude = extend_doe(new_doeX, possible_blade_number, new_altitude)

n_rot_val = np.array(new_doeX[:, 0])
D_val = np.array(new_doeX[:, 1])
RHO_val = np.array(new_doeX[:, 2])
v_val = np.array(new_doeX[:, 3])
PITCH_REF_val = np.array(new_doeX[:, 4])
pitch_val = np.array(new_doeX[:, 5])
c_mid_ratio_val = np.array(new_doeX[:, 6])
c_mid_radius_ratio_val = np.array(new_doeX[:, 7])
twist_tip = np.array(new_doeX[:, 8])
c_root_ratio_val = np.array(new_doeX[:, 9])
n_blade_val = np.array(new_doeX[:, 10])

print(new_doeX.shape)
print(new_altitude.shape)

In [ ]:
print(len(n_rot_val))
print(np.min(pitch_val), np.max(pitch_val))
print(len(new_altitude))

In [ ]:
print(np.min(n_rot_val), np.max(n_rot_val))
print(np.min(D_val), np.max(D_val))
print(np.min(RHO_val), np.max(RHO_val))
print(np.min(v_val), np.max(v_val))
print(np.min(c_mid_ratio_val), np.max(c_mid_ratio_val))
print(np.min(c_mid_radius_ratio_val), np.max(c_mid_radius_ratio_val))
print(np.min(PITCH_REF_val), np.max(PITCH_REF_val))
print(np.min(pitch_val), np.max(pitch_val))
print(np.min(twist_tip), np.max(twist_tip))
print(np.min(c_root_ratio_val), np.max(c_root_ratio_val))
print(np.min(n_blade_val), np.max(n_blade_val))

In [ ]:
import os.path as pth
from fastga_he.models.propulsion.components.propulsor.propeller.methodology.performance_point_d_var import (
    ComputePropellerPointPerformanceDVar,
)
from fastga.command.api import generate_block_analysis

# Define relative path
DATA_FOLDER_PATH = "data"
DATA_FILE = pth.join(DATA_FOLDER_PATH, "input_propeller_vplm.xml")

# Define the wing primary geometry parameters name as a list
var_inputs = [
    "data:geometry:propeller:average_rpm",
    "data:geometry:propeller:diameter",
    "data:geometry:propeller:blades_number",
    "data:geometry:propeller:mid_chord_ratio",
    "data:geometry:propeller:root_chord_ratio",
    "data:geometry:propeller:mid_chord_radius_ratio",
    "data:geometry:propeller:twist_tip",
    "data:aerodynamics:propeller:point_performance:rho",
    "data:aerodynamics:propeller:point_performance:twist_75",
    "data:aerodynamics:propeller:point_performance:twist_75_ref",
    "data:aerodynamics:propeller:point_performance:speed",
]

# Declare function
compute_propeller = generate_block_analysis(
    ComputePropellerPointPerformanceDVar(),
    var_inputs,
    str(DATA_FILE),
    overwrite=False,
)

In [ ]:
thrust_array = np.zeros_like(n_rot_val)
power_array = np.zeros_like(n_rot_val)
eta_array = np.zeros_like(n_rot_val)
sigma_array = np.zeros_like(n_rot_val)
AF_array = np.zeros_like(n_rot_val)
lambda_array = np.zeros_like(n_rot_val)

for idx, n_rot_it in enumerate(n_rot_val):
    inputs_dict = {
        "data:geometry:propeller:average_rpm": (n_rot_it, "rps"),
        "data:geometry:propeller:diameter": (D_val[idx], "m"),
        "data:geometry:propeller:blades_number": (n_blade_val[idx], None),
        "data:geometry:propeller:mid_chord_ratio": (c_mid_ratio_val[idx], None),
        "data:geometry:propeller:root_chord_ratio": (c_root_ratio_val[idx], None),
        "data:geometry:propeller:mid_chord_radius_ratio": (c_mid_radius_ratio_val[idx], None),
        "data:geometry:propeller:twist_tip": (twist_tip[idx], "rad"),
        "data:aerodynamics:propeller:point_performance:rho": (RHO_val[idx], "kg/m**3"),
        "data:aerodynamics:propeller:point_performance:twist_75": (pitch_val[idx], "rad"),
        "data:aerodynamics:propeller:point_performance:twist_75_ref": (PITCH_REF_val[idx], "rad"),
        "data:aerodynamics:propeller:point_performance:speed": (v_val[idx], "m/s"),
    }
    output_dict = compute_propeller(inputs_dict)
    thrust = output_dict["data:aerodynamics:propeller:point_performance:thrust"][0][0]
    power = output_dict["data:aerodynamics:propeller:point_performance:power"][0][0]
    eta = output_dict["data:aerodynamics:propeller:point_performance:efficiency"][0][0]
    sigma = output_dict["data:geometry:propeller:solidity_ratio"][0][0]
    AF = output_dict["data:geometry:propeller:activity_factor"][0][0]
    aspect_ratio = output_dict["data:geometry:propeller:aspect_ratio"][0][0]
    thrust_array[idx] = thrust
    power_array[idx] = power
    eta_array[idx] = eta
    sigma_array[idx] = sigma
    AF_array[idx] = AF
    lambda_array[idx] = aspect_ratio

    if idx % 10 == 0:
        print(idx, eta)

In [ ]:
from functools import reduce

# proper_result_idx = np.where(abs(eta_array - 0.5) < 0.5)[0]
proper_result_idx_1 = np.where(eta_array != np.nan)
proper_result_idx_2 = np.where(thrust_array > 0.0)
proper_result_idx_3 = np.where(power_array > 0.0)
proper_result_idx_4 = np.where(eta_array < 1.0)

proper_result_idx = reduce(
    np.intersect1d,
    (
        proper_result_idx_1,
        proper_result_idx_2,
        proper_result_idx_3,
        proper_result_idx_4,
    ),
)

print(proper_result_idx)

thrust_array = thrust_array[proper_result_idx]
power_array = power_array[proper_result_idx]
eta_array = eta_array[proper_result_idx]
sigma_array = sigma_array[proper_result_idx]
AF_array_test = copy.deepcopy(AF_array)
AF_array = AF_array[proper_result_idx]
lambda_array_test = copy.deepcopy(lambda_array)
lambda_array = lambda_array[proper_result_idx]

n_rot_val = n_rot_val[proper_result_idx]
D_val = D_val[proper_result_idx]
n_blade_val = n_blade_val[proper_result_idx]
twist_tip = twist_tip[proper_result_idx]
RHO_val = RHO_val[proper_result_idx]
pitch_val = pitch_val[proper_result_idx]
PITCH_REF_val = PITCH_REF_val[proper_result_idx]
v_val = v_val[proper_result_idx]

altitude = new_altitude[proper_result_idx]

ct_array = thrust_array / (RHO_val * n_rot_val ** 2 * D_val ** 4)
cp_array = power_array / (RHO_val * n_rot_val ** 3 * D_val ** 5)

In [ ]:
print(len(altitude))
print(len(ct_array))

In [ ]:
import pandas as pd

print(np.min(n_rot_val), np.max(n_rot_val))
print(np.min(D_val), np.max(D_val))
print(np.min(RHO_val), np.max(RHO_val))
print(np.min(v_val), np.max(v_val))
print(np.min(c_mid_ratio_val), np.max(c_mid_ratio_val))
print(np.min(c_mid_radius_ratio_val), np.max(c_mid_radius_ratio_val))
print(np.min(n_blade_val), np.max(n_blade_val))
print(np.min(twist_tip), np.max(twist_tip))
print(np.min(eta_array), np.max(eta_array))

results_stack = np.c_[
    thrust_array,
    power_array,
    eta_array,
    sigma_array,
    AF_array,
    lambda_array,
    n_rot_val,
    D_val,
    n_blade_val,
    twist_tip,
    RHO_val,
    pitch_val,
    PITCH_REF_val,
    v_val,
    altitude,
]
results_dataframe = pd.DataFrame(
    results_stack,
    columns=[
        "Thrust",
        "Power",
        "Efficiency",
        "Solidity",
        "Activity Factor",
        "Aspect ratio",
        "Rotational speed",
        "Diameter",
        "Number of blades",
        "Blade twist between root and tip",
        "Density",
        "Pitch at 0.75R",
        "Ref Pitch at 0.75R",
        "Flight speed",
        "Altitude",
    ],
)
file_name = "data/doe_fixed_pitch.csv"
results_dataframe.to_csv(file_name)

In [ ]:
results = pd.read_csv("data/full_doe.csv")
thrust_array = results["Thrust"].to_numpy()
power_array = results["Power"].to_numpy()
eta_array = results["Efficiency"].to_numpy()
sigma_array = results["Solidity"].to_numpy()
AF_array = results["Activity Factor"].to_numpy()
lambda_array = results["Aspect ratio"].to_numpy()
n_rot_val = results["Rotational speed"].to_numpy()
D_val = results["Diameter"].to_numpy()
n_blade_val = results["Number of blades"].to_numpy()
RHO_val = results["Density"].to_numpy()
pitch_val = results["Pitch at 0.75R"].to_numpy()
PITCH_REF_val = results["Ref Pitch at 0.75R"].to_numpy()
v_val = results["Flight speed"].to_numpy()
altitude = results["Altitude"].to_numpy()

print(np.min(n_rot_val), np.max(n_rot_val))
print(np.min(D_val), np.max(D_val))
print(np.min(RHO_val), np.max(RHO_val))
print(np.min(v_val), np.max(v_val))
print(np.min(c_mid_ratio_val), np.max(c_mid_ratio_val))
print(np.min(c_mid_radius_ratio_val), np.max(c_mid_radius_ratio_val))
print(np.min(n_blade_val), np.max(n_blade_val))

# Trying $\eta$ as a function of Ct and J

In [ ]:
# Import pyVPLM modules/functions
from pyvplm.addon.variablepowerlaw import regression_models, perform_regression

import pandas as pd

atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = eta_array
pi2 = ct_array
pi3 = v_val / (n_rot_val * D_val)
pi4 = (
    v_val ** 2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound ** 2.0
pi5 = v_val * D_val / atm.kinematic_viscosity
pi6 = sigma_array
pi7 = AF_array
pi8 = lambda_array
doe_eta = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8]
doe_eta = pd.DataFrame(doe_eta, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7", "pi8"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_eta.values, elected_pi0="pi1", order=2, log_space=True, ymax_axis=10, test_mode=True
)
perform_regression(doe_eta.values, model, choosen_model=20)

# Trying Ct as a function of J, twist and other parameters

In [ ]:
atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = ct_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val ** 2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound ** 2.0
pi4 = v_val * D_val / atm.kinematic_viscosity
pi5 = sigma_array
pi6 = pitch_val / PITCH_REF_val
pi7 = AF_array
pi8 = lambda_array
doe_ct = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8]
doe_ct = pd.DataFrame(doe_ct, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7", "pi8"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_ct.values, elected_pi0="pi1", order=3, log_space=True, ymax_axis=10, test_mode=True
)
perform_regression(doe_ct.values, model, choosen_model=40)

In [ ]:
print(np.round(ct_array[:5], 3))

In [ ]:
print(np.min(pi2), np.max(pi2))
print(np.min(pi3), np.max(pi3))
print(np.min(pi4 / 1e6), np.max(pi4 / 1e6))
print(np.min(pi5), np.max(pi5))
print(np.min(pitch_val / PITCH_REF_val), np.max(pitch_val / PITCH_REF_val))
print(np.min(pi7), np.max(pi7))
print(np.min(pi8), np.max(pi8))

# Trying Ct as a function of J, Cp and other parameters

In [ ]:
atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = ct_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val ** 2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound ** 2.0
pi4 = v_val * D_val / atm.kinematic_viscosity
pi5 = sigma_array
pi6 = cp_array
pi7 = AF_array
pi8 = lambda_array
doe_ct = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8]
doe_ct = pd.DataFrame(doe_ct, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7", "pi8"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_ct.values, elected_pi0="pi1", order=3, log_space=True, ymax_axis=10, test_mode=True
)
perform_regression(doe_ct.values, model, choosen_model=40)

In [ ]:
idx = np.where(altitude == 0.0)[0]
print(np.unique(D_val))

# Trying Cp as a function of J, Ct and other parameters

In [ ]:
atm = Atmosphere(altitude, altitude_in_feet=False)

pi1 = cp_array
pi2 = v_val / (n_rot_val * D_val)
pi3 = (
    v_val ** 2.0 + (n_rot_val * 2.0 * np.pi) ** 2.0 * (D_val / 2.0) ** 2.0
) / atm.speed_of_sound ** 2.0
pi4 = v_val * D_val / atm.kinematic_viscosity  # Somehow reduced the deviation :O
pi5 = sigma_array
pi6 = ct_array
pi7 = AF_array
pi8 = lambda_array
doe_cp = np.c_[pi1, pi2, pi3, pi4, pi5, pi6, pi7, pi8]
doe_cp = pd.DataFrame(doe_cp, columns=["pi1", "pi2", "pi3", "pi4", "pi5", "pi6", "pi7", "pi8"])

# Fit with 3rd order power-law model the obtained Pi DOE
model = regression_models(
    doe_cp.values, elected_pi0="pi1", order=3, log_space=True, ymax_axis=10, test_mode=True
)
perform_regression(doe_cp.values, model, choosen_model=40)

In [ ]:
print(pi2[0])
print(pi3[0])
print(pi4[0])
print(pi5[0])
print(pi6[0])
print(pi7[0])
print(pi8[0])

print(pi1[0])

In [ ]:
effy = pi6 * pi2 / pi1
print(np.max(effy), np.min(effy))